# Weather data assembly notebook
    # I'll import all data, assemble it, and run some preliminary graphing to find out a little bit about it

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import openpyxl

## Assemble the dataframe

In [3]:
# Let's get all of our files
df = pd.read_csv('.../data/raw-weather-data/3151369.csv')
df.to_csv('full_weather_database.csv', index=False)
df = pd.read_csv('full_weather_database.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw-weather-data/3151369.csv'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          4687 non-null   object 
 1   NAME             4687 non-null   object 
 2   DATE             4687 non-null   object 
 3   AWND             4474 non-null   float64
 4   AWND_ATTRIBUTES  4474 non-null   object 
 5   PRCP             4504 non-null   float64
 6   PRCP_ATTRIBUTES  4504 non-null   object 
 7   PSUN             1004 non-null   float64
 8   PSUN_ATTRIBUTES  1004 non-null   object 
 9   SNOW             4687 non-null   float64
 10  SNOW_ATTRIBUTES  4687 non-null   object 
 11  SNWD             4687 non-null   float64
 12  SNWD_ATTRIBUTES  4687 non-null   object 
 13  TAVG             2130 non-null   float64
 14  TAVG_ATTRIBUTES  2130 non-null   object 
 15  TMAX             4474 non-null   float64
 16  TMAX_ATTRIBUTES  4474 non-null   object 
 17  TMIN          

## Let's make our data more readable, feature engineer, and trim

In [5]:
# We can get rid of the STATION feature since we know this set is from DIA. WESD is almost entirely null so let's get rid of it. There are some other categories that we just won't need because the data is either duplicated, or worthless for our use.

In [6]:
# So we need to clean missing values, and fix the temperatures which are missing for a decent portion of the dataset. I'll need to get another dataset to do that. Should I feature engineer the temperature variance for each day? That could be interesting. If there's a high variance maybe snow is more likely to melt?
# We also need to rename some of these columns for clarity. Let's do that first. You could skip this step if you're happy with how they are.

In [7]:
def trim_and_rename(passed_df, column_list):
    rename_dict = {
                    'AWND': 'AVG_WIND_SPEED',
                    'PRCP': 'PRECIPITATION',
                    'SNOW': 'SNOWFALL',
                    'SNWD': 'SNOW_DEPTH',
                    'TAVG': 'AVG_TEMP',
                    'WSF2': 'WIND_2MIN_FASTEST',
                    'WSF5': 'WIND_5MIN_FASTEST',
                    'WT01': 'WT_FOG',
                    'WT02': 'WT_HEAVY_FOG',
                    'WT03': 'WT_THUNDER',
                    'WT04': 'WT_SMALL_HAIL',
                    'WT05': 'WT_HAIL',
                    'WT06': 'WT_FROST',
                    'WT07': 'WT_BLOWING_DUST',
                    'WT08': 'WT_SMOKE_OR_HAZE',
                    'WT09': 'WT_DRIFTING_SNOW',
                    'WT10': 'WT_TORNADO',
                    'WT11': 'WT_HIGH_WINDS',
                   # 'WT12': Doesn't exist in this dataset
                    'WT13': 'WT_MIST',
                    'WT14': 'WT_DRIZZLE',
                    'WT15': 'WT_FREEZING_DRIZZLE',
                    'WT16': 'WT_RAIN',
                    'WT17': 'WT_FREEZING_RAIN',
                    'WT18': 'WT_SNOW',
                    'WT19': 'WT_UNK_PRECIPITATION',
                   # 'WT20': Doesn't exist at all
                   # 'WT21': Doesn't exist in this dataset
                    'WT22': 'WT_FREEZING_FOG',
                    'TMAX': 'TEMP_MAX',
                    'TMIN': 'TEMP_MIN'
                    }
    passed_df['DATE'] = pd.to_datetime(passed_df['DATE'])
    passed_df = passed_df[passed_df.columns.drop(list(passed_df.filter(regex='_ATTRIBUTES')))]
    passed_df = passed_df.drop(column_list, axis=1)
    passed_df.rename(columns=rename_dict, inplace=True)

    return passed_df

In [8]:
df = trim_and_rename(df, column_list=['STATION', 'WESD', 'NAME', 'PSUN', 'TSUN'])

In [9]:
# Let's see if we can add average temperature to the missing columns using temp max and mins
def add_avg_temps(passed_df):
    beginning_num = passed_df['AVG_TEMP'].isna().sum()
    for row in passed_df.index:
        if np.isnan(passed_df.at[row, 'AVG_TEMP']):
            if not np.isnan(passed_df.at[row, 'TEMP_MAX']):
                if not np.isnan(passed_df.at[row, 'TEMP_MIN']):
                    min_temp = passed_df.at[row, 'TEMP_MIN']
                    max_temp = passed_df.at[row, 'TEMP_MAX']
                    passed_df.at[row, 'AVG_TEMP'] = (min_temp + max_temp) / 2
    ending_num = passed_df['AVG_TEMP'].isna().sum()
    total = str(beginning_num - ending_num)
    print('Added ' + total + ' values to AVG_TEMP.')

    return passed_df

In [10]:
df = add_avg_temps(df)

Added 2344 values to AVG_TEMP.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DATE                  4687 non-null   datetime64[ns]
 1   AVG_WIND_SPEED        4474 non-null   float64       
 2   PRECIPITATION         4504 non-null   float64       
 3   SNOWFALL              4687 non-null   float64       
 4   SNOW_DEPTH            4687 non-null   float64       
 5   AVG_TEMP              4474 non-null   float64       
 6   TEMP_MAX              4474 non-null   float64       
 7   TEMP_MIN              4474 non-null   float64       
 8   WIND_2MIN_FASTEST     4474 non-null   float64       
 9   WIND_5MIN_FASTEST     4462 non-null   float64       
 10  WT_FOG                1127 non-null   float64       
 11  WT_HEAVY_FOG          359 non-null    float64       
 12  WT_THUNDER            650 non-null    float64       
 13  WT_SMALL_HAIL     

In [12]:
# Let's see where the sets of NaN data are....
#
 # for row in df.index:
 #    if np.isnan(df.at[row, 'AVG_TEMP']):
 #        print(row)

In [13]:
# Starts at row 2282, 2013-04-01 ends at 2494, 2013-11-30. Multiple features during this timeframe are missing values. We'll need to fill these. My first thought is to use another dataset from a close by station in order to replicate the original data. We're also coming back up here to add in the 2020 data at the same time.

In [14]:
df_additional = pd.read_csv('.../data/raw-weather-data/3160654.csv')
df_additional2 = pd.read_csv('.../data/raw-weather-data/3161094.csv')

In [15]:
# It looks like Central Park is the only set with a complete reading for the timeframe. That'll work.

In [16]:
df_additional = df_additional.loc[df_additional['NAME'] == 'DENVER CENTRAL PARK, CO US']

In [17]:
df_additional = trim_and_rename(df_additional, column_list=['STATION', 'WESD', 'NAME'])
df_additional2 = trim_and_rename(df_additional2, column_list=['STATION', 'NAME'])

In [18]:
df_additional = add_avg_temps(df_additional)
df_additional2 = add_avg_temps(df_additional2)

Added 273 values to AVG_TEMP.
Added 29 values to AVG_TEMP.


In [19]:
def combine_df(big_df, little_df, third_df):
    for row in big_df.index:
        if np.isnan(big_df.at[row, 'TEMP_MAX']):
            big_df.drop(index=row, inplace=True)

    little_df['DATE'] = pd.to_datetime(little_df['DATE'])
    big_df = pd.concat([big_df, little_df, third_df])
    big_df.sort_values(by=['DATE'], axis=0, inplace=True)
    big_df.reset_index(drop=True, inplace=True)

    return big_df

In [20]:
df = combine_df(df, df_additional, df_additional2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5114 entries, 0 to 5113
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DATE                  5114 non-null   datetime64[ns]
 1   AVG_WIND_SPEED        4840 non-null   float64       
 2   PRECIPITATION         5114 non-null   float64       
 3   SNOWFALL              5114 non-null   float64       
 4   SNOW_DEPTH            5114 non-null   float64       
 5   AVG_TEMP              5113 non-null   float64       
 6   TEMP_MAX              5113 non-null   float64       
 7   TEMP_MIN              5113 non-null   float64       
 8   WIND_2MIN_FASTEST     4840 non-null   float64       
 9   WIND_5MIN_FASTEST     4828 non-null   float64       
 10  WT_FOG                1207 non-null   float64       
 11  WT_HEAVY_FOG          379 non-null    float64       
 12  WT_THUNDER            733 non-null    float64       
 13  WT_SMALL_HAIL     

In [21]:
# Now lets take care of the rest of our NaN values.

df.fillna(0, inplace=True)

In [22]:
df.to_csv('weather_cleaned.csv', index=False)

In [23]:
# After looking at the crash data it looks like some of our dates are off. We can add all of 2020 weather data into the set. I'll go back up and import that.